In [1]:
!pip install imageio
!pip install torch torchvision

    48% |███████████████▌                | 233.9MB 36.6MB/s eta 0:00:07

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b0d0000 @  0x7f8e934c91c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 19.1MB/s 
    100% |████████████████████████████████| 2.0MB 12.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os, tarfile
import imageio
from scipy.io import loadmat
# from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

import tensorflow as tf
print(tf.test.gpu_device_name())

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms

/device:GPU:0


In [0]:
FER_URL = 'https://www.dropbox.com/s/uv14ve5uky71i99/fer2013.csv?dl=1'

# Utility functions

#### ToDos
- Create a function to fetch data from a url.
- Check if it is already downloaded.
- Check if the file is csv or tar gz etc.
- Add cross-validation code to be able to use sklearn cross_val_score function to quickly evaluate the performance.

In [0]:
def fetch_data(URL, DOWNLOAD_FOLDER, DOWNLOAD_FILE):
  if not os.path.isdir(DOWNLOAD_FOLDER):
   os.makedirs(DOWNLOAD_FOLDER)
  
  if not os.path.isfile(DOWNLOAD_FOLDER+DOWNLOAD_FILE):
    print('Beginning file download...')
    urllib.request.urlretrieve(URL, DOWNLOAD_FOLDER+DOWNLOAD_FILE)
    print('Done.')
  

In [0]:
def split_train_test(XY, n_splits=1, test_size=0.2, random_state=42):
    split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    for train_index, test_index in split.split(XY[0], XY[1]):
        X_train, Y_train = XY[0][train_index,:], XY[1][train_index]
        X_test, Y_test = XY[0][test_index,:], XY[1][test_index]
        
    return X_train, Y_train, X_test, Y_test

In [0]:
def get_fer_data(url, download_folder, download_file, split_data=False):
    
    fetch_data(url, download_folder, download_file)
    df = pd.read_csv(download_folder+download_file)
    Y = df['emotion'].as_matrix()
    X_str = df['pixels'].as_matrix()
    X = []
    for row in X_str:
        X.append(np.fromstring(row, dtype=int, sep=' '))

    X = np.array(X) / 255.0
    
    #X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)

    usage = df['Usage'].as_matrix()
    X_train, Y_train = X[usage=='Training'], Y[usage=='Training']
    X_test, Y_test = X[(usage=='PrivateTest') | (usage=='PublicTest')], Y[(usage=='PrivateTest') | (usage=='PublicTest')]

    if split_data:
        return split_train_test((X_new, Y), n_splits=1, test_size=0.2, random_state=42)

    return X_train, Y_train, X_test, Y_test

In [0]:
def one_hot_encoder(label):
    encoder = OneHotEncoder(dtype=np.float32)
    label_1hot = encoder.fit_transform(label.reshape(-1,1))
    print('The labels are: {}'.format(np.unique(label)))
    return label_1hot

# Load data

In [0]:
root_folder = 'drive/app/fer/'
# root_folder = 'D:/dev/data/'

In [9]:
X_train, Y_train, X_test, Y_test = get_fer_data(FER_URL, root_folder, 'fer2013.csv',
                                                split_data=False)

print("Train: [{}, {}], Test: [{}, {}]".format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))

Train: [(28709, 2304), (28709,)], Test: [(7178, 2304), (7178,)]


In [10]:
!ls -l drive/app/fer

total 294020
-rw-r--r-- 1 root root 301072766 May 16 18:51 fer2013.csv


In [11]:
Y_train_1hot = one_hot_encoder(Y_train).toarray().view(np.float32)
Y_test_1hot = one_hot_encoder(Y_test).toarray().view(np.float32)
# print(Y_train_1hot[0:2])
# print(type(Y_train_1hot))

The labels are: [0 1 2 3 4 5 6]
The labels are: [0 1 2 3 4 5 6]


# Model definition

In [0]:
class CNN(nn.Module):
    def __init__(self, width, height, n_channels, n_classes):
        super(CNN, self).__init__()

        self.width = width
        self.height = height
        self.channels = n_channels
        self.conv_1 = nn.Conv2d(in_channels=n_channels, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.fc5 = nn.Linear(in_features=32*12*12, out_features=1024)
        self.fc5 = nn.Linear(in_features=64*24*24, out_features=1024)
        self.fc6 = nn.Linear(in_features=1024, out_features=n_classes)
        self.activation = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(p=0.5)
        self.dropout2d = nn.Dropout2d(p=0.5)
    
    
    def forward(self, X):
    
        out = X.view(-1, self.channels, self.width, self.height)
        out = self.conv_1(out)
        out = self.dropout2d(out)
        out = self.activation(out)
        out = self.conv_2(out)
        out = self.activation(out)
        #print(out.size())
        out = self.maxpool(out)
        #print(out.size())
        out = self.conv_3(out)
        out = self.dropout2d(out)
        out = self.activation(out)
        out = self.conv_4(out)
        out = self.activation(out)
        #print(out.size())
        out = self.maxpool(out)
        #print(out.size())
        out = out.view(out.size(0), -1)
        out = self.fc5(out)
        out = self.dropout(out)
        out = self.activation(out)
        out = self.fc6(out)

        return out
  
  
    def fit(self, X, Y, criterion, optimizer, epochs, n_batches, batch_size, print_time, X_test=None, Y_test=None):
    
        X = torch.from_numpy(X).double()
        Y = torch.from_numpy(Y).long()

        train_data = torch.utils.data.TensorDataset(X, Y)
        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

        iteration = 0
        for epoch in range(epochs):
            for i, (x, y) in enumerate(train_loader):
                if torch.cuda.is_available():
                    x = Variable(x.cuda())
                    y = Variable(y.cuda())
                else:
                    x = Variable(x)
                    y = Variable(y)

                optimizer.zero_grad()
                outputs = self.forward(x)
                loss = criterion(outputs, y)
                loss.backward()
                optimizer.step()

                iteration += 1

                if iteration%print_time == 0:
                    if X_test is not None:
                        predictions = self.predict(X_test, X_test.shape[0]//batch_size, batch_size)
                        accuracy = np.round(self.score(Y_test, predictions), 2)
                        print('Epoch: {}, Iteration: {}, Loss: {}, Test accuracy: {}%'.format(epoch, iteration, loss, accuracy))
                    else:
                        print('Epoch: {}, Iteration: {}, Loss: {}'.format(epoch, iteration, loss))
                    #accuracy = predict(test_loader)
          
          
    def predict(self, X, n_batches, batch_size):
    
        correct = 0
        test_cost = 0
        total = 0
        first = True
        for ibatch in range(n_batches):
            X_batch = torch.from_numpy(X[ibatch*batch_size:(ibatch+1)*batch_size,:])

            outputs = self.forward(X_batch)
            if first == True:
                predicted = torch.argmax(outputs, dim=1)
                first = False
            else:
                predicted = torch.cat((predicted, torch.argmax(outputs, dim=1)))
                
        # Remaining images left in the batch
        X_batch = torch.from_numpy(X[(ibatch+1)*batch_size:,:])
        outputs = self.forward(X_batch)
        predicted = torch.cat((predicted, torch.argmax(outputs, dim=1)))

        return predicted
  
    def score(self, Y, predicted):
    
        #predicted = torch.argmax(predicted, axis=1)
        accuracy = 100*(Y == predicted.data.numpy()).sum()/Y.shape[0]
        return accuracy
        

In [0]:
WIDTH = 48
HEIGHT = 48
N_CHANNELS = 1
N_CLASSES = 7
BATCH_SIZE =64
MAX_ITER = 3

In [14]:
ann = CNN(WIDTH, HEIGHT, N_CHANNELS, N_CLASSES)
ann = ann.double()
if torch.cuda.is_available():
    ann.cuda()
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(ann.parameters(), lr=0.01)

RuntimeError: ignored

In [16]:
for parameter in list(ann.parameters()):
    print(parameter.size())

torch.Size([32, 1, 5, 5])
torch.Size([32])
torch.Size([64, 32, 5, 5])
torch.Size([64])
torch.Size([128, 64, 5, 5])
torch.Size([128])
torch.Size([256, 128, 5, 5])
torch.Size([256])
torch.Size([1024, 36864])
torch.Size([1024])
torch.Size([7, 1024])
torch.Size([7])


In [19]:
N_BATCHES = X_train.shape[0]//BATCH_SIZE
PRINT_TIME = N_BATCHES//(N_BATCHES)
TEST_N_BATCHES = X_test.shape[0]//BATCH_SIZE


ann.fit(X_train, np.squeeze(Y_train), cost_fn, optimizer, MAX_ITER, N_BATCHES, BATCH_SIZE, PRINT_TIME, X_test, Y_test)

Epoch: 0, Iteration: 1, Loss: 1.9436706484702633, Test accuracy: 22.65%
Epoch: 1, Iteration: 2, Loss: 1.9375704131565172, Test accuracy: 22.64%
Epoch: 2, Iteration: 3, Loss: 1.92872980627995, Test accuracy: 23.08%


In [0]:
# !kill -9 -1